In [1]:
# import the necessary packages
from imutils.video import VideoStream
from imutils.video import FPS
import imutils
import time
import cv2
from typing import Tuple



class OpenCVTracker:
    OPENCV_OBJECT_TRACKERS = {
        "csrt": cv2.TrackerCSRT_create,
        "kcf": cv2.TrackerKCF_create,
        "boosting": cv2.TrackerBoosting_create,
        "mil": cv2.TrackerMIL_create,
        "tld": cv2.TrackerTLD_create,
        "medianflow": cv2.TrackerMedianFlow_create,
        "mosse": cv2.TrackerMOSSE_create
    }
    def __init__(self, trackerType):
        self.trackerType = trackerType
        self.__trackers = {}
        
    def track(self, uniqueId: str, image, bbox: Tuple[int, int, int, int] = None) -> Tuple[int, int, int, int]:
        if bbox is not None:
            tr = self.OPENCV_OBJECT_TRACKERS[self.trackerType]()
            self.__trackers[uniqueId] = tr
            return tr.init(image, bbox)
        else:
            return self.__trackers[uniqueId].update(image)


tracker = OpenCVTracker("medianflow")

# initialize the bounding box coordinates of the object we are going
# to track
ids = []
# if a video path was not supplied, grab the reference to the web cam
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(1.0)


[INFO] starting video stream...


In [2]:
fps = None
currentMaxId = 0
# loop over frames from the video stream
while True:
    # grab the current frame, then handle if we are using a
    # VideoStream or VideoCapture object
    frame = vs.read()
#     frame = frame[1] if args.get("video", False) else frame

    # check to see if we have reached the end of the stream
    if frame is None:
        break

    # resize the frame (so we can process it faster) and grab the
    # frame dimensions
    frame = imutils.resize(frame, width=500)
    (H, W) = frame.shape[:2]
        # check to see if we are currently tracking an object
    if len(ids) > 0:
        # grab the new bounding box coordinates of the object
        for thisId in ids:
            (success, box) = tracker.track(thisId, frame)

            # check to see if the tracking was a success
            if success:
                (x, y, w, h) = [int(v) for v in box]
                cv2.rectangle(frame, (x, y), (x + w, y + h),
                    (0, 255, 0), 2)

        # update the FPS counter
        fps.update()
        fps.stop()

        # initialize the set of information we'll be displaying on
        # the frame
        info = [
            ("Tracker", tracker.trackerType),
            ("Success", "Yes" if success else "No"),
            ("FPS", "{:.2f}".format(fps.fps())),
        ]

        # loop over the info tuples and draw them on our frame
        for (i, (k, v)) in enumerate(info):
            text = "{}: {}".format(k, v)
            cv2.putText(frame, text, (10, H - ((i * 20) + 20)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the 's' key is selected, we are going to "select" a bounding
    # box to track
    if key == ord("s"):
        # select the bounding box of the object we want to track (make
        # sure you press ENTER or SPACE after selecting the ROI)
        initBB = cv2.selectROI("Frame", frame, fromCenter=False,
            showCrosshair=True)
        currentMaxId = currentMaxId + 1
        # start OpenCV object tracker using the supplied bounding box
        # coordinates, then start the FPS throughput estimator as well
        tracker.track(str(currentMaxId), frame, initBB)
        ids.append(str(currentMaxId))
        fps = FPS().start()
            # if the `q` key was pressed, break from the loop
    elif key == ord("q"):
        break

vs.stop()
# close all windows
cv2.destroyAllWindows()